In [1]:
%pip install pydot
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, Input, Activation, Dropout
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [2]:
df = pd.read_csv("dataset_clean.csv", index_col= 0)

In [3]:
df

,label,clean_text
0,1,ABOARD AIR FORCE ONE (Reuters) - President Bar...
1,1,WASHINGTON (Reuters) - U.S. Supreme Court Tues...
2,1,WASHINGTON (Reuters) - President Donald Trump’...
3,0,mainstream media salivating indictment Paul Ma...
4,1,HANOI (Reuters) - court Vietnam Friday upheld ...
...,...,...
65096,0,Rapper T. I. unloaded black celebrities met Do...
65097,0,Green Bay Packers lost Washington Redskins Wee...
65098,0,Macy’s today grew union several great names Am...
65099,1,"NATO, Russia Hold Parallel Exercises Balkans 1..."


In [4]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(df["clean_text"], df["label"],test_size=0.2)


In [7]:
max_words = 2000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train.astype(str))
sequences = tok.texts_to_sequences(X_train.astype(str))
sequences_matrix = pad_sequences(sequences,maxlen=max_len)
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)

    return model


In [8]:
model = RNN()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=7, validation_split=0.2, callbacks=[tensorboard_callback])


Epoch 1/7


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


326/326 [==============================] - 24s 70ms/step - loss: 0.2631 - accuracy: 0.8931 - val_loss: 0.3801 - val_accuracy: 0.8439
Epoch 2/7
326/326 [==============================] - 23s 69ms/step - loss: 0.1641 - accuracy: 0.9404 - val_loss: 0.1832 - val_accuracy: 0.9329
Epoch 3/7
326/326 [==============================] - 23s 70ms/step - loss: 0.1326 - accuracy: 0.9532 - val_loss: 0.1463 - val_accuracy: 0.9422
Epoch 4/7
326/326 [==============================] - 23s 70ms/step - loss: 0.1141 - accuracy: 0.9598 - val_loss: 0.1451 - val_accuracy: 0.9488
Epoch 5/7
326/326 [==============================] - 23s 70ms/step - loss: 0.1005 - accuracy: 0.9641 - val_loss: 0.1711 - val_accuracy: 0.9339
Epoch 6/7
326/326 [==============================] - 23s 70ms/step - loss: 0.0904 - accuracy: 0.9690 - val_loss: 0.1561 - val_accuracy: 0.9374
Epoch 7/7
326/326 [==============================] - 23s 72ms/step - loss: 0.0798 - accuracy: 0.9729 - val_loss: 0.1441 - val_accuracy: 0.9552


In [18]:
model.build(input_shape=(100,1))
model.summary()
plot_model(model, to_file='lstm_model.png', show_shapes=True, show_layer_names=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [11]:
%load_ext tensorboard


In [12]:
# model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
#           validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [13]:
test_sequences = tok.texts_to_sequences(X_test.astype(str))
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [14]:
y_pred = model.predict(test_sequences_matrix)
y_pred_binary = (y_pred > 0.5).astype(int)


from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(Y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')

# Calculate and print classification report
classification_rep = classification_report(Y_test, y_pred_binary)
print(f'Classification Report:\n{classification_rep}')

# Calculate and print precision, recall, and F1 score
precision = precision_score(Y_test, y_pred_binary)
recall = recall_score(Y_test, y_pred_binary)
f1 = f1_score(Y_test, y_pred_binary)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

407/407 [==============================] - 4s 9ms/step
Accuracy: 0.9472390753398356
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      6871
           1       0.96      0.93      0.94      6150

    accuracy                           0.95     13021
   macro avg       0.95      0.95      0.95     13021
weighted avg       0.95      0.95      0.95     13021

Precision: 0.9591527987897126
Recall: 0.9278048780487805
F1 Score: 0.9432184478056039
